In [1]:
from autocnet.graph import network
from autocnet.examples import get_path, available

In [2]:
available()

/Users/jlaura/github/autocnet/autocnet/examples/


['.DS_Store',
 'Apollo15',
 'ChemCam',
 'LIBS_TEST',
 'logging.json',
 'logging.yaml',
 'lunar_lamb.svg',
 'lunar_lamb_wkt.svg',
 'Lunar_LRO_LOLA_Shade_MAP2_90.0N20.0_LAMB.tif',
 'mars_merc.svg',
 'mars_merc_wkt.svg',
 'Mars_MGS_MOLA_ClrShade_MAP2_0.0N0.0_MERC.tif',
 'Mars_MGS_MOLA_ClrShade_MAP2_90.0N0.0_POLA.tif',
 'mars_polar.svg',
 'mars_polar_wkt.svg',
 'SP_2C_02_02358_S138_E3586.jpg',
 'SP_2C_02_02358_S138_E3586.spc',
 'SP_2C_02_03860_S136_E3557.spc']

In [3]:
available('Apollo15')

/Users/jlaura/github/autocnet/autocnet/examples/Apollo15


['AS15-M-0296_SML.png',
 'AS15-M-0297_SML.png',
 'AS15-M-0298_SML.png',
 'AS15-M-0299_SML.png',
 'AS15-M-0300_SML.png']

In [ ]:
for i in range(296, 301):
    p = get_path('AS15-M-0{}_SML.png'.format(i))
    print(p)

In [ ]:
G = network.CandidateGraph()